In [85]:
import os
import pandas as pd
import numpy as np
import numpy 
import sys
from collections import *
pd.options.mode.chained_assignment = None
from tqdm import tqdm

In [28]:
os.chdir('/Users/DennisLin/Documents/Python/CSE258/final/kkbox_recommender/')
data_path = './kkbox-music-recommendation-challenge/'
song_path = os.path.join(data_path, 'songs.csv')
song_info = os.path.join(data_path, 'song_extra_info.csv')
train_path = os.path.join(data_path, 'train.csv')
test_path = os.path.join(data_path, 'test.csv')

In [3]:
ls kkbox-music-recommendation-challenge/


members.csv               song_extra_info.csv       test.csv
members.csv.7z            song_extra_info.csv.7z    test.csv.7z
sample_submission.csv     songs.csv                 train.csv
sample_submission.csv.7z  songs.csv.7z              train.csv.7z


### Members

In [4]:
class Members():
    def __init__(self, path):
        self.members = pd.read_csv(path)
        self.preprocess()
    
    def preprocess(self):
        '''
        1. remove wierd age
        2. replace registers -> years
        3. add using date
        4. gender to three class
        '''
        #print info
        self.info()
        
        # 1
        wierd_age = list()
        for age, count in Counter(self.members['bd']).items():
            if count < 10:
                wierd_age.append(age)
        for drop_age in wierd_age:
            self.members = self.members[self.members['bd'] != drop_age]        
        
        #2 & 3
        self.members['registration_init_time'] = pd.to_datetime(self.members['registration_init_time'], format='%Y%m%d')
        self.members['expiration_date'] = pd.to_datetime(self.members['expiration_date'], format='%Y%m%d')
        self.members = self.members[self.members['expiration_date'] != '1970-01-01 00:00:00']
        self.members['activation_days'] = self.members['expiration_date'] - self.members['registration_init_time']
        self.members['activation_days'].astype(int)
        self.members['expiration_date'] = pd.DatetimeIndex(self.members['expiration_date']).year
        self.members['expiration_date'] -= min(self.members['expiration_date'])
        self.members['registration_init_time'] = pd.DatetimeIndex(self.members['registration_init_time']).year
        self.members['registration_init_time'] -= min(self.members['registration_init_time'])


        
        #4    
        self.members['gender'][self.members['gender'].isnull()] = 0
        self.members['gender'][self.members['gender'] == 'male'] = 1
        self.members['gender'][self.members['gender'] == 'female'] = -1
        
        #print info
        self.info()


    
    def info(self):
        print("The number of total users is: {}".format(len(self.members)))

In [5]:
members = Members(os.path.join(data_path, 'members.csv'))

The number of total users is: 34403
The number of total users is: 34288


### Songs

In [6]:
class clean_song_data():
    def __init__(self, song, song_info):
        self.song = pd.read_csv(song)
        self.song_info = pd.read_csv(song_info)
        self.result = pd.merge(self.song, self.song_info, on='song_id')
        self.filter_1()
        self.filter_2()
        self.add_count()
        self.convert_type()
        
    def filter_1(self):
        '''
        1. Filter out NaN
        2. Use only English songs
        
        '''
        header = []
        self.result = self.result.dropna()
#         for i in self.result.columns:
#             header.append(i)
#         for i in header:
#             self.result = self.result[pd.notnull(self.result[i])]
        
        # 52.0 = English
        self.result = self.result[self.result.language == 52.0]
    
    def filter_2(self):
        '''
        genre and artist count > 15
        
        '''
        header = ['genre_ids','artist_name']
        for i in header:
            k = pd.DataFrame(self.result.groupby(i).size().items())
            k = k.rename(columns={0: 'item', 1: 'count'})
            k = k[k['count']>15]
            self.result = self.result[self.result[i].isin(k['item'])]
            self.result.head()
            print(i,len(self.result))
            
    def add_count(self):
        '''
        add count for genre, composer 
        
        '''
        header = ['genre_ids', 'composer']
        name = ['gener_count', 'composer_count']
        for num,i in enumerate(header):
            l = []
            for j in self.result[i]:
                l.append(len(j.split('|')))
            self.result[name[num]] = l
            
    def convert_type(self):
        '''
        Convert str to list
        '''
        tmp = []
        for i in self.result['genre_ids'].tolist():
            tmp.append(list(map(int,i.split('|'))))
        self.result['genre_ids_list'] = tmp

In [10]:
song = clean_song_data(song_path, song_info)

genre_ids 168873
artist_name 121316


In [12]:
song.result

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language,name,isrc,gener_count,composer_count,genre_ids_list
9,btcG03OHY3GNKWccPP0auvtSbhxog/kllIIOx5grE/k=,232629,352|1995,Kodaline,Stephen Garrigan| Mark Prendergast| Vincent Ma...,Stephen Garrigan| Mark Prendergast| Vincent Ma...,52.0,The One,GBARL1401580,2,5,"[352, 1995]"
35,NV9HhUzyK50tGvxb3w0PdZoaw3Ypp86XDmmMr0vgFdg=,262749,139,John Legend,John Stephens,John Stephens,52.0,Under the Stars,USSM11509174,1,1,[139]
39,OLnB6amnCTMEz5KxrV7pdkXJWB+fnMoWVJSzv2AlBLk=,236355,465,Westlife,Bettis| Hammond,Bettis| Hammond,52.0,When You Tell Me That You Love Me,GBARL0500645,1,2,[465]
87,UfK2UdQAaYF6IJUXqSJ10FsbAF3YkvMJYcTYC2Ec5m4=,222649,1609,Calvin Harris,Calvin Harris,Calvin Harris,52.0,Summer,GBARL1400296,1,1,[1609]
103,nO+tjL6/qHgNO/ui0TjeP+nSsD+aoGcT9AHU7Dtrzsc=,227327,880,Casting Crowns,Mark Hall|Matt Maher|John Mabe|Blake Bollinger,Mark Hall|Matt Maher|John Mabe|Blake Bollinger,52.0,You Are the Only One,USA5W1300245,1,4,[880]
...,...,...,...,...,...,...,...,...,...,...,...,...
2295357,fetIp2IHaquy4808NCWZ3e6zUWPAAauySCC/o6/1oL0=,204242,893,Andy Williams,Richard Rodgers / Oscar Hammerstein II,Richard Rodgers / Oscar Hammerstein II,52.0,The Surrey with the Fringe on Top,DEPZ69063557,1,1,[893]
2295360,PiTvnLMAEcL5OTdi8InPZbB/ShOXv0J0Kb6KCDdY9pc=,220656,698,Various Artists,Wham,George Michael,52.0,Last Christmas,DEAR41212908,1,1,[698]
2295375,G4CFe4RcqGuNFxZ+kxPrjsafx8+bI5kbjqy1pcKMyCY=,113684,698,Various Artists,G.Autry| O.Haldeman,G.Autry| O.Haldeman,52.0,Here Comes Santa Claus,CAM460407867,1,2,[698]
2295396,LIBVFt9mqBUtZu911th7hIuQ6ecZJEZYr3k8drsqEGI=,309404,465,Mecano,Ignacio Cano,Ignacio Cano,52.0,La Fuerza del Destino,ES5028800029,1,1,[465]


### Train

In [15]:
train = pd.read_csv(train_path)

In [22]:
len(train)

7377418

In [36]:
test = pd.read_csv(test_path)
len(test)

2556790

In [17]:
valid_user = set(members.members['msno'])
valid_song = set(song.result['song_id'])

In [20]:
train_user = train[train['msno'].isin(valid_user)]

In [23]:
train_filtered = train_user[train_user['song_id'].isin(valid_song)]

In [24]:
len(train_filtered)

322383

In [32]:
class Data():
    def __init__(self, path1, path2, songs, users):
        self.train = pd.read_csv(path1)
        self.test = pd.read_csv(path2)
        self.songs = songs
        self.users = users
        
        
        self.train = self.filterd(self.train)
        self.test = self.filterd(self.test)
        
        
        self.people2songs = self.train.groupby('msno')["song_id"].apply(list)
        self.songs2count = self.train.groupby('song_id').size()
        self.song2user = self.train.groupby("song_id")["msno"].apply(list)
#         self.songs_set = set(self.songs['song_id'])
        self.all = pd.concat([self.train, self.test], axis=0, sort=False)
        self.people2songs_all = self.all.groupby('msno')["song_id"].apply(set)
    
    def filterd(self, pd):
        valid_user = set(self.users.members['msno'])
        valid_song = set(self.songs.result['song_id'])
        pd = pd[pd['msno'].isin(valid_user)]
        pd = pd[pd['song_id'].isin(valid_song)]
        return pd
        
def sample(positive_data, all_songs, people2songs):
    negative_dict = dict()
    for people, count in tqdm(positive_data.groupby('msno').size().items()):
        negative_dict[people] = random.sample(all_songs - people2songs[people], count)
    append_list = []
    for people, songs in negative_dict.items():
        for song in songs:
            append_list.append([people, song, 0])
    positive_matrix = np.hstack((positive_data.values[:, :2], np.ones((positive_data.shape[0], 1))))
    return np.vstack((positive_matrix, np.array(append_list)))

In [33]:
data = Data(train_path, test_path, song, members)

In [35]:
print(len(data.train))
print(len(data.test))

322383
100528


In [105]:
class Model():
    def __init__(self, train):
        self.train = train
        self.pop = self.gen_pop()
    def gen_pop(self, percentile=0.5):
        counts = 0
        out_set = set()
        vc= self.train['song_id'].value_counts()
        for song, count in vc.items():
            out_set.add(song)
            counts += count
            if counts > len(data.train)*percentile:
                break
        return out_set
    
    def baseline(self, song):
        if song in self.pop:
            return 1
        else:
            return 0
        
    
        

In [106]:
model = Model(data.train)

In [78]:
data.test.iloc[0]['song_id']

'f0Yvo22LVUbRIlICXcEEym7IYNVjUqghZryP6Vf103A='

In [101]:
matrix = data.test.values[:, 2]
songs = list(matrix)


In [102]:
songs

['...',
 ...]

In [107]:
out = []

for song in tqdm(songs):
    out.append(model.baseline(song))




100%|██████████| 100528/100528 [00:00<00:00, 1007312.73it/s]


In [68]:
len(vc)

19043